In [1]:
!pip install mnist 
import numpy as np
import mnist
from tensorflow import keras

You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [2]:
# The first time you run this might be a bit slow, since the
# mnist package has to download and cache the data.
train_images = mnist.train_images()
train_labels = mnist.train_labels()

print(train_images.shape) # (60000, 28, 28)
print(train_labels.shape) # (60000,)

(60000, 28, 28)
(60000,)


1- normalize the image pixel values from [0, 255] to [-0.5, 0.5] 
to make our network easier to train (using smaller, centered values usually leads to better results) 
2-reshape each image from (28, 28) to (28, 28, 1) because Keras requires the third dimension.

In [3]:
import numpy as np
import mnist

train_images = mnist.train_images()
train_labels = mnist.train_labels()
test_images = mnist.test_images()
test_labels = mnist.test_labels()

# Normalize the images.
train_images = (train_images / 255) - 0.5
test_images = (test_images / 255) - 0.5

# Reshape the images.
train_images = np.expand_dims(train_images, axis=3)
test_images = np.expand_dims(test_images, axis=3)

print(train_images.shape) # (60000, 28, 28, 1)
print(test_images.shape)  # (10000, 28, 28, 1)

(60000, 28, 28, 1)
(10000, 28, 28, 1)


The Sequential constructor takes an array of Keras Layers. We’ll use 3 types of layers for our CNN: Convolutional, Max Pooling, and Softmax.

* num_filters, filter_size, and pool_size are self-explanatory variables that set the hyperparameters for our CNN.
* The first layer in any Sequential model must specify the input_shape, so we do so on Conv2D. Once this input shape is specified, Keras will automatically infer the shapes of inputs for later layers.
* The output Softmax layer has 10 nodes, one for each class.

In [4]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten
from tensorflow.keras.models import Sequential
num_filters = 8
filter_size = 3
pool_size = 2

model = Sequential([
  Conv2D(num_filters, filter_size, input_shape=(28, 28, 1)),
  MaxPooling2D(pool_size=pool_size),
  Flatten(),
  Dense(10, activation='softmax'),
])

In [5]:
model.compile(
  'adam',
  loss='categorical_crossentropy',
  metrics=['accuracy'],
)

In [6]:
import mnist

train_labels = mnist.train_labels()
print(train_labels[0]) # 5

5


In [7]:
from tensorflow.keras.utils import to_categorical

model.fit(
  train_images,
  to_categorical(train_labels),
  epochs=3,
  validation_data=(test_images, to_categorical(test_labels)),
)

Epoch 1/3
1875/1875 [==============================] - 13s 7ms/step - loss: 0.3547 - accuracy: 0.8971 - val_loss: 0.2282 - val_accuracy: 0.9314
Epoch 2/3
1875/1875 [==============================] - 12s 7ms/step - loss: 0.1937 - accuracy: 0.9434 - val_loss: 0.1551 - val_accuracy: 0.9560
Epoch 3/3
1875/1875 [==============================] - 12s 7ms/step - loss: 0.1363 - accuracy: 0.9614 - val_loss: 0.1128 - val_accuracy: 0.9673


In [8]:
model.save_weights('cnn.h5')

In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten

num_filters = 8
filter_size = 3
pool_size = 2

# Build the model.
model = Sequential([
  Conv2D(num_filters, filter_size, input_shape=(28, 28, 1)),
  MaxPooling2D(pool_size=pool_size),
  Flatten(),
  Dense(10, activation='softmax'),
])

# Load the model's saved weights.
model.load_weights('cnn.h5')

In [10]:
# Predict on the first 5 test images.
predictions = model.predict(test_images[:5])

# Print our model's predictions.
print(np.argmax(predictions, axis=1)) # [7, 2, 1, 0, 4]

# Check our predictions against the ground truths.
print(test_labels[:5]) # [7, 2, 1, 0, 4]

[7 2 1 0 4]
[7 2 1 0 4]


Another code 

In [11]:
import numpy as np
from matplotlib import pyplot as plt

from keras import backend as K
# from keras.models import Sequential
from keras.layers import Input, Dense, Dropout, Activation, ZeroPadding2D
from keras.layers import Flatten, Conv2D, MaxPooling2D, BatchNormalization
from keras.models import Model, load_model, model_from_json, model_from_yaml
from keras.utils import to_categorical

batch_sz = 128
n_classes = 10
n_epoch = 12
shapeX = (28,28,1)
classes = np.asarray([0,1,2,3,4,5,6,7,8,9],dtype = np.float32)

In [12]:
def MNIST_Model(input_shape = (28,28,1),classes = 10):
	X_input = Input(input_shape)

	# zero padding probably not required since the main digit is in the centre only
	# X = zeroPadding2D((1,1))(X_input)

	X = Conv2D(32,(3,3),strides = (1,1), name = 'conv0')(X_input)
	X = BatchNormalization(axis=3,name='bn0')(X)
	X = Activation('relu')(X)
	X = Conv2D(32,(3,3),strides = (1,1), name = 'conv1')(X)
	X = BatchNormalization(axis=3,name='bn1')(X)
	X = Activation('relu')(X)
	X = MaxPooling2D((2,2),strides = (2,2),name = 'MP1')(X)

	X = Conv2D(64,(3,3),strides = (1,1), name = 'conv2')(X)
	X = BatchNormalization(axis=3,name='bn2')(X)
	X = Activation('relu')(X)
	X = Conv2D(64,(3,3),strides = (1,1), name = 'conv3')(X)
	X = BatchNormalization(axis=3,name='bn3')(X)
	X = Activation('relu')(X)
	X = MaxPooling2D((2,2),strides = (2,2),name = 'MP2')(X)
	
	X = Dropout(0.2)(X)
	X = Flatten()(X)
	X = Dense(256,activation = 'relu',name= 'fc1')(X)
	X = Dropout(0.4)(X)
	X = Dense(n_classes,activation = 'softmax',name = 'fco')(X)

	model = Model(inputs = X_input,outputs = X, name = 'MNIST_Model')
	return model


In [13]:

modelMNIST = MNIST_Model(shapeX,n_classes)
print (modelMNIST.summary())

modelMNIST.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics=['accuracy'])

modelMNIST.fit(train_images, to_categorical(train_labels), epochs = n_epoch, batch_size = batch_sz)

pred = modelMNIST.evaluate(test_images,to_categorical(test_labels))




Model: "MNIST_Model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv0 (Conv2D)               (None, 26, 26, 32)        320       
_________________________________________________________________
bn0 (BatchNormalization)     (None, 26, 26, 32)        128       
_________________________________________________________________
activation (Activation)      (None, 26, 26, 32)        0         
_________________________________________________________________
conv1 (Conv2D)               (None, 24, 24, 32)        9248      
_________________________________________________________________
bn1 (BatchNormalization)     (None, 24, 24, 32)        128       
_________________________________________________________________
activation_1 (Activation)    (None, 24, 24, 32)        

In [14]:
print ("Loss = " + str(pred[0]))
print ("Test Accuracy = " + str(pred[1]))

Loss = 0.023071179166436195
Test Accuracy = 0.9930999875068665


In [15]:

modelMNIST.save_weights('cnn_1.h5')


In [16]:
# Predict on the first 5 test images.
predictions = modelMNIST.predict(test_images[:5])

# Print our model's predictions.
print(np.argmax(predictions, axis=1)) # [7, 2, 1, 0, 4]

# Check our predictions against the ground truths.
print(test_labels[:5]) # [7, 2, 1, 0, 4]

[7 2 1 0 4]
[7 2 1 0 4]


In [17]:
# Predict on the first 5 test images.
predictions = modelMNIST.predict(test_images[:7])

# Print our model's predictions.
print(np.argmax(predictions, axis=1)) # [7 2 1 0 4 1 4]

# Check our predictions against the ground truths.
print(test_labels[:7]) # [7 2 1 0 4 1 4]

[7 2 1 0 4 1 4]
[7 2 1 0 4 1 4]
